In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "../src/modeling")

import model
import metrics

import os
print(os.listdir("."))

Using TensorFlow backend.


['Processed.csv', 'Preprocessing.ipynb', 'Modeling.ipynb', '.ipynb_checkpoints']
